In [1]:
import pickle
import os

os.listdir()

['.DS_Store',
 'results-06-10-2020--16:21:57.pickle',
 '2020-10-06-explore-evaluation.ipynb',
 'relevant_optimus_code',
 'sample_text.txt',
 'latents.pickle',
 'reconstructed_sentences.pickle',
 '.ipynb_checkpoints',
 'autoregressive_versus_teacher_forcing.py',
 'data',
 'outputs',
 '2020-09-28-explore_optimus_code.ipynb']

In [3]:
with open('results-06-10-2020--16:21:57.pickle', 'rb') as handle:
    results = pickle.load(handle)

In [5]:
for snli_wikipedia in results.keys():
    for beta, r in results[snli_wikipedia].items():
        for k, v in r.items():
            print(k, v.shape)

pred_auto_all torch.Size([200, 9])
ce_losses_auto_all torch.Size([200])
pred_tf_all torch.Size([200, 9])
ce_losses_tf_all torch.Size([200])
pred_auto_all torch.Size([400, 9])
ce_losses_auto_all torch.Size([400])
pred_tf_all torch.Size([400, 9])
ce_losses_tf_all torch.Size([400])
pred_auto_all torch.Size([600, 9])
ce_losses_auto_all torch.Size([600])
pred_tf_all torch.Size([600, 9])
ce_losses_tf_all torch.Size([600])
pred_auto_all torch.Size([800, 9])
ce_losses_auto_all torch.Size([800])
pred_tf_all torch.Size([800, 9])
ce_losses_tf_all torch.Size([800])
pred_auto_all torch.Size([1000, 9])
ce_losses_auto_all torch.Size([1000])
pred_tf_all torch.Size([1000, 9])
ce_losses_tf_all torch.Size([1000])
